# Поглавље 7: Израда чет апликација
## Брзи почетак са OpenAI API-јем

Овај нотебук је прилагођен из [Azure OpenAI репозиторијума са примерима](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) који садржи нотебуке за приступ [Azure OpenAI](notebook-azure-openai.ipynb) сервисима.

Python OpenAI API ради и са Azure OpenAI моделима, уз неколико измена. Више о разликама можете сазнати овде: [Како да пребацујете између OpenAI и Azure OpenAI ендпоинта у Python-у](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# Преглед  
„Велики језички модели су функције које пресликавају текст у текст. Када добију улазни низ текста, велики језички модел покушава да предвиди који ће текст следећи доћи“(1). Овај „брзи водич“ ће корисницима представити основне концепте великих језичких модела, кључне захтеве пакета за почетак рада са AML-ом, лагани увод у дизајн упита, као и неколико кратких примера различитих начина употребе.


## Садржај  

[Преглед](../../../../07-building-chat-applications/python)  
[Како користити OpenAI сервис](../../../../07-building-chat-applications/python)  
[1. Креирање вашег OpenAI сервиса](../../../../07-building-chat-applications/python)  
[2. Инсталација](../../../../07-building-chat-applications/python)    
[3. Акредитиви](../../../../07-building-chat-applications/python)  

[Примери употребе](../../../../07-building-chat-applications/python)    
[1. Сажимање текста](../../../../07-building-chat-applications/python)  
[2. Класификација текста](../../../../07-building-chat-applications/python)  
[3. Генерисање нових имена производа](../../../../07-building-chat-applications/python)  
[4. Фино подешавање класификатора](../../../../07-building-chat-applications/python)  

[Референце](../../../../07-building-chat-applications/python)


### Направите свој први промпт  
Ова кратка вежба ће вам дати основни увод у слање промптова OpenAI моделу за једноставан задатак „сажимање“.


**Кораци**:  
1. Инсталирајте OpenAI библиотеку у своје Python окружење  
2. Учитајте стандардне помоћне библиотеке и подесите своје уобичајене OpenAI безбедносне креденцијале за OpenAI сервис који сте креирали  
3. Одаберите модел за свој задатак  
4. Направите једноставан промпт за модел  
5. Пошаљите свој захтев на API модела!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. Проналажење одговарајућег модела  
GPT-3.5-turbo или GPT-4 модели могу да разумеју и генеришу природни језик.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. Дизајн промпта  

„Магија великих језичких модела је у томе што, тренирајући се да минимизују грешку у предвиђању на огромним количинама текста, модели на крају науче појмове који су корисни за ова предвиђања. На пример, они науче појмове као што су“(1):

* како се пише
* како функционише граматика
* како се парафразира
* како се одговара на питања
* како се води разговор
* како се пише на више језика
* како се програмира
* итд.

#### Како контролисати велики језички модел  
„Од свих улаза у велики језички модел, далеко највећи утицај има текстуални промпт(1).

Велики језички модели могу се подстаћи да генеришу излаз на неколико начина:

Инструкција: Реците моделу шта желите
Довршавање: Наведите модел да доврши почетак онога што желите
Демонстрација: Покажите моделу шта желите, било са:
Неколико примера у самом промпту
Много стотина или хиљада примера у датасету за фино подешавање“



#### Постоје три основне смернице за креирање промптова:

**Покажите и реците.** Јасно ставите до знања шта желите, било инструкцијама, примерима или комбинацијом оба. Ако желите да модел поређа листу ставки по абецедном реду или да класификује пасус по сентименту, покажите му да је то оно што желите.

**Обезбедите квалитетне податке.** Ако покушавате да направите класификатор или да модел прати неки образац, уверите се да има довољно примера. Обавезно проверите своје примере — модел је обично довољно паметан да препозна основне правописне грешке и да вам ипак да одговор, али може и да претпостави да је то намерно и то може утицати на одговор.

**Проверите своја подешавања.** Подешавања temperature и top_p одређују колико ће модел бити детерминистички у генерисању одговора. Ако тражите одговор где постоји само један тачан одговор, онда би требало да их подесите ниже. Ако желите разноврсније одговоре, можда ћете желети да их подесите више. Најчешћа грешка коју људи праве са овим подешавањима је што мисле да су то контроле за „памет“ или „креативност“.


Извор: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Сажми текст  
#### Изазов  
Сажмите текст тако што ћете додати 'tl;dr:' на крај пасуса. Обратите пажњу како модел разуме како да изврши више задатака без додатних упутстава. Можете експериментисати са описнијим упитима од tl;dr да бисте променили понашање модела и прилагодили сажетак који добијате(3).  

Недавна истраживања су показала значајан напредак у многим NLP задацима и бенчмарковима кроз претходно тренирање на великом корпусу текста, а затим фино подешавање за одређени задатак. Иако је овај приступ обично независан од задатка по архитектури, и даље захтева специфичне скупова података за фино подешавање са хиљадама или десетинама хиљада примера. Насупрот томе, људи углавном могу да изврше нови језички задатак са само неколико примера или једноставних упутстава – што је нешто са чим се савремени NLP системи и даље углавном муче. Овде показујемо да повећање обима језичких модела значајно побољшава независне перформансе са мало примера, понекад чак достижући ниво конкурентности са претходним најбољим приступима фино подешавања.

Tl;dr


# Вежбе за различите случајеве употребе  
1. Сажми текст  
2. Класификуј текст  
3. Генериши нова имена производа


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Класификуј текст  
#### Изазов  
Класификуј ставке у категорије које се задају у тренутку извршавања. У следећем примеру, у упиту дајемо и категорије и текст који треба класификовати (*playground_reference).

Кориснички упит: Здраво, један тастер на тастатури мог лаптопа се недавно поломио и треба ми замена:

Класификована категорија:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Генериши нова имена производа
#### Изазов
Смисли имена производа на основу датих речи. Овде у упутству наводимо информације о производу за који треба да смислимо имена. Такође дајемо сличан пример да покажемо какав образац желимо да добијемо. Поставили смо и вредност temperature високо како бисмо добили насумичније и креативније одговоре.

Опис производа: Аппарат за прављење милкшејкова код куће  
Кључне речи: брзо, здраво, компактно.  
Имена производа: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Опис производа: Пар ципела које могу да пристају свакој величини стопала.  
Кључне речи: прилагодљиво, пристаје, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Референце  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Примери](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Најбоље праксе за фино подешавање GPT-3 за класификацију текста](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# За додатну помоћ  
[OpenAI Комерцијални тим](AzureOpenAITeam@microsoft.com)


# Сарадници
* Louis Li



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални људски превод. Не сносимо одговорност за било каква неспоразума или погрешна тумачења настала коришћењем овог превода.
